#### WISDOMAIN 데이터 기반 특허 분석 실습

##### 1. 데이터 준비

In [61]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import statements required for Plotly 
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# Import and suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [62]:
# WISDOMAIN 원본 데이터 확인
rawdata = pd.read_csv('../data/rawdata_q2_2.csv')
rawdata.head(1)

,번호,명칭,요약,등록일,출원인,출원인국가,독립 청구항수,전체 청구항수,명칭(원문),요약(원문),...,통상실시권자,발명자국가,공통특허분류,미국특허분류,소유권이전여부,소유권최종양도일,현재소유자,권리 현황,최종 상태,권리만료예상일
0,US9607236,Method and apparatus for providing loan verifi...,Some aspects of the invention relate to a mobi...,2017.03.28,"Blinker, Inc.",US,5.0,30,NaN,NaN,...,NaN,US|US|US,"G06V20/63, G06Q40/025, G06T11/60, G06V10/267, ...",001/001,N,NaN,BLINKER INC,Alive,Alive,2034.06.27


In [63]:
# 컬럼명 확인
rawdata.columns

Index(['번호', '명칭', '요약', '등록일', '출원인', '출원인국가', '독립 청구항수', '전체 청구항수', '명칭(원문)',
       '요약(원문)', '국제특허분류', '자국인용특허', '외국인용특허', '자국피인용특허', '자국인용횟수', '자국피인용횟수',
       '특허평가등급', '발명자수', '발명자', '출원인대표명', '출원인주소', '전용실시권자', '통상실시권자', '발명자국가',
       '공통특허분류', '미국특허분류', '소유권이전여부', '소유권최종양도일', '현재소유자', '권리 현황', '최종 상태',
       '권리만료예상일'],
      dtype='object')

In [64]:
# 필요한 컬럼들 정의
target_cols = ['번호','명칭','요약','등록일','출원인','독립 청구항수','전체 청구항수',
               '국제특허분류','자국인용특허', '외국인용특허', '자국피인용특허', '자국인용횟수', 
               '자국피인용횟수','특허평가등급', '발명자수','발명자국가','출원인대표명']
dataset = rawdata[target_cols]
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839 entries, 0 to 838
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   번호       839 non-null    object 
 1   명칭       839 non-null    object 
 2   요약       839 non-null    object 
 3   등록일      392 non-null    object 
 4   출원인      735 non-null    object 
 5   독립 청구항수  838 non-null    float64
 6   전체 청구항수  839 non-null    int64  
 7   국제특허분류   839 non-null    object 
 8   자국인용특허   458 non-null    object 
 9   외국인용특허   150 non-null    object 
 10  자국피인용특허  219 non-null    object 
 11  자국인용횟수   510 non-null    float64
 12  자국피인용횟수  510 non-null    float64
 13  특허평가등급   382 non-null    object 
 14  발명자수     839 non-null    int64  
 15  발명자국가    839 non-null    object 
 16  출원인대표명   735 non-null    object 
dtypes: float64(3), int64(2), object(12)
memory usage: 111.6+ KB
None


In [65]:
# 연도가 없는 데이터 삭제
dataset.dropna(subset = ["등록일"], inplace=True)

In [66]:
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 394
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   번호       392 non-null    object 
 1   명칭       392 non-null    object 
 2   요약       392 non-null    object 
 3   등록일      392 non-null    object 
 4   출원인      375 non-null    object 
 5   독립 청구항수  392 non-null    float64
 6   전체 청구항수  392 non-null    int64  
 7   국제특허분류   392 non-null    object 
 8   자국인용특허   333 non-null    object 
 9   외국인용특허   112 non-null    object 
 10  자국피인용특허  147 non-null    object 
 11  자국인용횟수   333 non-null    float64
 12  자국피인용횟수  333 non-null    float64
 13  특허평가등급   382 non-null    object 
 14  발명자수     392 non-null    int64  
 15  발명자국가    392 non-null    object 
 16  출원인대표명   375 non-null    object 
dtypes: float64(3), int64(2), object(12)
memory usage: 55.1+ KB
None


In [67]:
def change_ipc(row):
    ipcs = [ipc.split('/')[0].split('-') for ipc in [item.strip() for item in row]]
    ipcs_revised = list(set([f"{each_ipc[0]}{each_ipc[1].replace('0','')}" for each_ipc in ipcs]))
    return ipcs_revised

def count_len(row):
    if type(row) == list:
        return len(row)
    else:
        return 0
    
def remove_duplicated_list(row):
    after_row = list(set(row))
    return after_row

def extract_year(row):
    return row.split('.')[0]

In [68]:
# 새로운 컬럼들 생성
# 1. 국가
dataset.loc[dataset['번호'].str.contains('KR'),'국가'] = 'KR'
dataset.loc[dataset['번호'].str.contains('US'),'국가'] = 'US'
# 2. 연도
dataset['연도'] = dataset['등록일'].apply(extract_year)

dataset.head(1)

,번호,명칭,요약,등록일,출원인,독립 청구항수,전체 청구항수,국제특허분류,자국인용특허,외국인용특허,자국피인용특허,자국인용횟수,자국피인용횟수,특허평가등급,발명자수,발명자국가,출원인대표명,국가,연도
0,US9607236,Method and apparatus for providing loan verifi...,Some aspects of the invention relate to a mobi...,2017.03.28,"Blinker, Inc.",5.0,30,"G06K-009/00, G06K-009/32, G06T-011/60, G06Q-04...","US3544771,US3550084,US5227803,US5579008,US5579...","CN103985256,CN204303027,HU03002998,JP10134219,...","US9888235,US9990726,US10445873,US10515285,US10...",593.0,17.0,A+,3,US|US|US,BLINKER INC,US,2017


In [69]:
dataset['국제특허분류'] = dataset['국제특허분류'].str.split(',')
dataset['외국인용횟수'] = dataset['외국인용특허'].str.split(',')
dataset['발명자국가'] = dataset['발명자국가'].str.split('|')

In [70]:
dataset['국제특허분류'] = dataset['국제특허분류'].apply(change_ipc)
dataset['외국인용횟수'] = dataset['외국인용횟수'].apply(count_len)
dataset['발명자국가'] = dataset['발명자국가'].apply(remove_duplicated_list)

In [71]:
dataset.head(1)

,번호,명칭,요약,등록일,출원인,독립 청구항수,전체 청구항수,국제특허분류,자국인용특허,외국인용특허,자국피인용특허,자국인용횟수,자국피인용횟수,특허평가등급,발명자수,발명자국가,출원인대표명,국가,연도,외국인용횟수
0,US9607236,Method and apparatus for providing loan verifi...,Some aspects of the invention relate to a mobi...,2017.03.28,"Blinker, Inc.",5.0,30,"[G06K9, G06T11, G06Q4]","US3544771,US3550084,US5227803,US5579008,US5579...","CN103985256,CN204303027,HU03002998,JP10134219,...","US9888235,US9990726,US10445873,US10515285,US10...",593.0,17.0,A+,3,[US],BLINKER INC,US,2017,12


In [72]:
dataset.iloc[0]['국제특허분류']

['G06K9', 'G06T11', 'G06Q4']